In [1]:
%load_ext tensorboard

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
from datetime import datetime

c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [3]:
def norm(array):
   return array/9

def invNorm(array):
   return 9*array

In [7]:
games = pd.read_csv("./data/sudoku.csv")

quizzes = games['quizzes']
solutions = games['solutions']

quizzes, solutions = shuffle(np.array(quizzes), np.array(solutions))


features = []
labels = []

fraction_used = 100

for i in range(int(len(quizzes)/fraction_used)):
   temp_arr = [norm(int(char)) for char in quizzes[i]]
   features.append(np.array(temp_arr).reshape((9,9,1)))

   if(i%200000==0):
       print(str(i/20000) + "%")



#We represent a 1 as label 0, 2 as label 1, etc.

for i in range(int(len(solutions)/fraction_used)):
   temp_arr = [(int(char)-1) for char in solutions[i]]
   labels.append(np.array(temp_arr).reshape((81,1)))
   if(i%200000==0):
       print(str(i/20000 +50)+ "%")


features = np.array(features)
labels = np.array(labels)
print("100%")
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

005060030000170490000903875203008000000040781100500200510409000020600008978000060 
 795864132832175496461923875243718659659342781187596243516489327324657918978231564
0.0%
50.0%
100%


In [4]:
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, kernel_size=(1,1), activation='relu', padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(81*9))
    model.add(layers.Reshape((81, 9)))
    model.add(layers.Activation('softmax'))
    return model

# model = create_model()
# model.summary()

In [5]:
model = create_model()
model.load_weights('./checkpoints/latest_checkpoint')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 128)         1280      
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 128)         512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          4160      
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 729)               3

In [6]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [7]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

  1/250 [..............................] - ETA: 0s - loss: 1.5530 - accuracy: 0.8218WARNING:tensorflow:From c:\users\user\pycharmprojects\tensorflowsudoku\venv\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
250/250 [==============================] - 11s 45ms/step - loss: 1.5619 - accuracy: 0.8087 - val_loss: 1.5630 - val_accuracy: 0.8073

In [13]:
    %tensorboard --logdir logs/scalars


Launching TensorBoard...

In [9]:
model.save_weights('checkpoints/latest_checkpoint')

In [10]:
def step_by_step(inputGame):

    finished = False
    while not finished:

        predictions = model.predict(inputGame.reshape((1,9,9,1)))[0]
        #Unnormalize the predictions and square values
        predict = np.argmax(predictions, axis=1).reshape((9,9))+1
        prob = np.max(predictions, axis=1).reshape((9,9))
        inputGame = invNorm(inputGame).reshape((9,9))
        missing = (inputGame==0)

        if missing.sum()==0:
            finished = True
        else:
            max = 0
            ind = 0
            for i in range(9):
                for j in range(9):
                    if missing[i][j] and prob[i][j]>= max:
                        max = prob[i][j]
                        ind = i*9 + j
                    elif missing[i][j] and prob[i][j]>0.8:
                        inputGame[i][j] = predict[i][j]
            inputGame[int(ind/9)][(ind%9)] = predict[int(ind/9)][(ind%9)]
            inputGame = norm(inputGame)

    return predict



def solve(game):
    game = norm(np.array([int(j) for j in game]).reshape((9,9,1)))
    game = step_by_step(game)
    return game

In [11]:
test_board = '004300209005009001070060043006002087190007400050083000600000105003508690042910300'

solved = solve(test_board)

print(solved)

[[8 7 4 3 7 1 2 5 9]
 [2 6 5 4 4 9 7 7 1]
 [9 7 1 5 6 5 8 4 3]
 [4 3 6 1 5 2 9 8 7]
 [1 9 8 6 5 7 4 3 2]
 [4 5 7 1 8 3 9 6 6]
 [6 8 9 2 3 4 1 7 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 2 8]]


In [12]:
print("End")





End
